In [19]:
import pandas as pd
import requests as r
from datetime import *     
import numpy as np
import os

In [20]:
from DataLoader import DataLoader

In [21]:
df = pd.read_csv('/Users/prasunbanerjee/Desktop/UniversityMaterials/Research/CEAR Hub/secoora/test_data.csv')
df = df.iloc[:, [0, 1]].reset_index(drop=True)
df.columns = ["time", "result"]
df = df.drop(0, axis=0)
df

,time,result
1,2024-12-25T17:50:10.81222Z,-2.098
2,2024-12-25T17:55:19.800911Z,-2.064
3,2024-12-25T18:00:34.410826Z,-2.036
4,2024-12-25T18:05:43.303884Z,-2.017
5,2024-12-25T18:10:57.930353Z,-2.001
...,...,...
647,2024-12-28T05:20:38.736164Z,-2.928
648,2024-12-28T05:21:05.116032Z,-2.927
649,2024-12-28T05:21:32.13423Z,-2.925
650,2024-12-28T05:26:45.716006Z,-2.911


In [22]:
df = DataLoader.transform_data(df)
df = DataLoader.order_df(df)
df = df.dropna()
df

,year,month,day,day_of_week,hour,minute,second,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3,result
4,2024,12,25,2,18,5,43,-1.000000,-1.836970e-16,0.974928,-0.222521,-2.449294e-16,1.0,-2.036,-2.064,-2.098,-2.039000,0.023643,-2.017
5,2024,12,25,2,18,10,57,-1.000000,-1.836970e-16,0.974928,-0.222521,-2.449294e-16,1.0,-2.017,-2.036,-2.064,-2.018000,0.017521,-2.001
6,2024,12,25,2,18,16,13,-1.000000,-1.836970e-16,0.974928,-0.222521,-2.449294e-16,1.0,-2.001,-2.017,-2.036,-2.000333,0.017010,-1.983
7,2024,12,25,2,18,21,27,-1.000000,-1.836970e-16,0.974928,-0.222521,-2.449294e-16,1.0,-1.983,-2.001,-2.017,-1.978000,0.025865,-1.95
8,2024,12,25,2,18,26,36,-1.000000,-1.836970e-16,0.974928,-0.222521,-2.449294e-16,1.0,-1.95,-1.983,-2.001,-1.948667,0.035019,-1.913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,2024,12,28,5,5,20,38,0.965926,2.588190e-01,-0.974928,-0.222521,-2.449294e-16,1.0,-2.93,-2.932,-2.932,-2.930000,0.002000,-2.928
648,2024,12,28,5,5,21,5,0.965926,2.588190e-01,-0.974928,-0.222521,-2.449294e-16,1.0,-2.928,-2.93,-2.932,-2.928333,0.001528,-2.927
649,2024,12,28,5,5,21,32,0.965926,2.588190e-01,-0.974928,-0.222521,-2.449294e-16,1.0,-2.927,-2.928,-2.93,-2.926667,0.001528,-2.925
650,2024,12,28,5,5,26,45,0.965926,2.588190e-01,-0.974928,-0.222521,-2.449294e-16,1.0,-2.925,-2.927,-2.928,-2.921000,0.008718,-2.911


In [31]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [37]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Last timestep output
        out = self.fc(out)
        return out

In [38]:
X = df.drop(columns=['result']).values
y = df['result'].values

In [39]:
sequence_length = 3

def create_sequences(X, y, seq_length):
    X_seq = []
    y_seq = []
    for i in range(seq_length, len(X)):
        X_seq.append(X[i-seq_length:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y, sequence_length)

# Manual train-test split
split_idx = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

In [42]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [43]:
input_size = X_seq.shape[2]
hidden_size = 64
num_layers = 2

In [44]:
LSTM_model = LSTMModel(input_size, hidden_size, num_layers)

In [45]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(LSTM_model.parameters(), lr=0.001)

In [46]:
num_epochs = 50

for epoch in range(num_epochs):
    LSTM_model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = LSTM_model(X_batch).squeeze()
        loss = criterion(outputs, y_batch.squeeze())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

Epoch [10/50], Loss: 2.9716
Epoch [20/50], Loss: 2.9412
Epoch [30/50], Loss: 2.9509
Epoch [40/50], Loss: 2.9663
Epoch [50/50], Loss: 2.9347
